<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Amun Inspection Workflow

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import sys

In [ ]:
USER, = !echo "$$USER"

sys.path.insert(0, f"/home/{USER}/code/amun-api/")
sys.path.insert(0, f"/home/{USER}/code/argo-python-client/")
sys.path.insert(0, f"/home/{USER}/code/common/")

---

In [ ]:
# Provided via https://github.com/thoth-station/amun-api

from amun.dockerfile import create_dockerfile

In [ ]:
from thoth.common.workflows import Workflow
from thoth.common.workflows import WorkflowManager

In [ ]:
from thoth.common.logging import init_logging

In [ ]:
init_logging({'thoth.common.workflows': 'INFO'})

---

In [ ]:
namespace = "macermak-thoth-dev"

In [ ]:
%env THOTH_AMUN_INFRA_NAMESPACE      $namespace 
%env THOTH_AMUN_INSPECTION_NAMESPACE $namespace
%env THOTH_WORKFLOW_CONTEXT          true

In [ ]:
workflow_manager = WorkflowManager(ocp_config={
    "kubernetes_verify_tls": False
})

---

In [ ]:
specification = {
  "base": "registry.access.redhat.com/ubi8/python-36",
  "build": {
    "requests": {
      "cpu": "500m",
      "hardware": {
        "cpu_family": 6,
        "cpu_model": 94,
        "physical_cpus": 32, "processor": "Intel-Xeon-Processor-Skylake-IBRS"
      },
      "memory": "1Gi"
    }
  },
  "identifier": "test-inspection",
  "python_packages": [
    "pipenv"
  ],
  "run": {
    "requests": {
      "cpu": "500m",
      "hardware": {
        "cpu_family": 6,
        "cpu_model": 94,
        "physical_cpus": 32,
        "processor": "Intel-Xeon-Processor-Skylake-IBRS"
      },
      "memory": "256Mi"
    }
  },
  "script": "#!/usr/bin/bash\necho \"Here should be run tests...\"\n",
}

In [ ]:
dockerfile, _ = create_dockerfile(specification)

In [ ]:
inspection_id = "test-inspection-v2"

template_parameters = {
    "AMUN_INSPECTION_ID": inspection_id,
    "THOTH_INFRA_NAMESPACE": namespace
}

workflow_parameters=dict(
    dockerfile=dockerfile,
    specification=json.dumps(specification)
)

In [ ]:
!argo delete --all

workflow_id = workflow_manager.submit_inspection_workflow(
    inspection_id,
    template_parameters=template_parameters,
    workflow_parameters=workflow_parameters,
)

workflow_id

In [ ]:
!argo watch $workflow_id

---